## Wordle Python implementation
Ahmad Basyouni, Alysa Vega, Miguel Luna

In [1]:
! pip install emoji

In [1]:
import pandas as pd
from collections import Counter
import emoji # ! pip install emoji

In [2]:
from src.file import get_simulation_results_folder
from src.pattern import (
    get_pattern,
    get_possible_words,
    pattern_to_int_list,
    patterns_to_string,
)
from src.prior import get_frequency_based_priors, get_true_wordle_prior, get_word_list
from src.solver import brute_force_optimal_guess, optimal_guess

In [3]:
allowed_words = get_word_list("wordle", short=True)
possible_words = get_word_list("wordle")

word_list = pd.DataFrame(allowed_words, columns=["word"], index=None)
word_list["day"] = [i for i in range(0, 2315)]

word_list.head()

,word,day
0,cigar,0
1,rebut,1
2,sissy,2
3,humph,3
4,awake,4


In [7]:
next_guess_map = {}

def get_next_guess(guesses, 
                   patterns, 
                   possibilities, 
                   priors, 
                   second_guess_map = None, 
                   hard_mode = False, 
                   brute_force_optimize = False, 
                   look_two_ahead = False,
                   purely_maximize_information = False,
                   optimize_for_uniform_distribution = False
                  ):
    
        phash = "".join(
            str(g) + "".join(map(str, pattern_to_int_list(p)))
            for g, p in zip(guesses, patterns, strict=True)
        )
    
        if second_guess_map is not None and len(patterns) == 1:
            next_guess_map[phash] = second_guess_map[patterns[0]]
        if phash not in next_guess_map:
            choices = possible_words
            if hard_mode:
                for guess, pattern in zip(guesses, patterns, strict=True):
                    choices = get_possible_words(guess, pattern, choices, game_name)
            if brute_force_optimize:
                next_guess_map[phash] = brute_force_optimal_guess(
                    choices,
                    possibilities,
                    priors,
                    game_name="wordle",
                    n_top_picks=brute_force_depth,
                )
            else:
                next_guess_map[phash] = optimal_guess(
                    choices,
                    possibilities,
                    priors,
                    "wordle",
                    look_two_ahead=look_two_ahead,
                    purely_maximize_information=purely_maximize_information,
                    optimize_for_uniform_distribution=optimize_for_uniform_distribution,
                )
        return next_guess_map[phash]

In [25]:
# Class that represents information about, and the ability to play a single game of Wordle
class game:
    
    # Initialize 
    def __init__(self): 
        self.set_game() # This sets game word and number
        self.results = []
        self.attempts = 0

    # Set game based on Wordle number
    def set_game(self, number = -1):    # Randomize by default

        if(number < 0 or number > 2313):     
            self.word = word_list.sample(n=1).iloc[0]["word"]
            self.date = word_list[word_list["word"] == self.word].iloc[0]["day"]
        else:
            self.word = word_list[word_list["day"] == number].iloc[0]["word"]   # Randomize on invalid number or input
            self.date = number

    def __str__(self):
        return f"{self.word}: guessed in {self.attempts} tries: \n{self.results}\n"
    
    # A game of Wordle
    def play(self):
        
        solved = False

        priors = get_frequency_based_priors("wordle")
        patterns = []
        possibility_counts = []
        possibilities = list(filter(lambda w: priors[w] > 0, possible_words))
        
        while (self.attempts < 6 and solved == False): # 6 tries

            counts = Counter(self.word) # Get counts of each character in the word
            guess = input(f"Enter a word for guess {self.attempts + 1}:").lower()
                
            if not(guess.isalpha()) or len(guess) != 5: # Invalid string (not a word)
                print("Invalid guess format. Guess must contain only alphabetical characters and be 5 characters long.")
            elif guess not in possible_words: # Word not in list of existing Wordle answers
                print("Word not recognized.")
                
            else:

                correctness = ""

                for i in range(0, 5): # Compare corresponding letter between guess and word

                    if guess[i] == self.word[i]:
                        correctness += ":green_square: " # Green square emoji
                        counts[guess[i]] -= 1
                        
                    elif guess[i] in self.word and counts[guess[i]] > 0:
                        correctness += ":yellow_square: " # Yellow square emoji
                        counts[guess[i]] -= 1
                        
                    else:
                        correctness += ":black_large_square: " # Black square emoji
                        

                print(guess, "\n", emoji.emojize(correctness)) # Print guess and results (as emojis)

                pattern = get_pattern(guess, self.word, "wordle")
                self.results.append(guess)
                patterns.append(pattern)
                possibilities = get_possible_words(guess, pattern, possibilities, "wordle")
                possibility_counts.append(len(possibilities))
                
                if(guess == self.word):
                    solved = True

                if not solved:
                    hint = get_next_guess(self.results, patterns, possibilities, priors)
                    # print("Patterns:", patterns)
                    print(f"{possibility_counts} possibilities:", possibilities)
                    print("Hint (most likely candidate or for maximizing information):", hint)
                
                self.attempts += 1

        if solved:
            congrats = ["Genius", "Magnificent", "Impressive", "Splendid", "Great", "Phew"]
            print(congrats[self.attempts - 1])  # Print corresponding congratulatory message like real Wordle
        else:
            self.attempts += 1          # Add attempt to make a loss count as 7 attempts
            print(self.word.upper())    # Reveal solution

In [26]:
new_game = game()
new_game.set_game(number=0) # Word is "cigar"
new_game.play()

Enter a word for guess 1: crane


crane 
 🟩 🟨 🟨 ⬛ ⬛ 
[35] possibilities: ['cigar', 'cargo', 'carry', 'coral', 'carol', 'cobra', 'circa', 'carat', 'caird', 'capri', 'carap', 'carbo', 'carbs', 'carby', 'cardi', 'cards', 'cardy', 'carks', 'carls', 'carob', 'carom', 'carpi', 'carps', 'carrs', 'carta', 'carts', 'carvy', 'cauri', 'cimar', 'copra', 'coram', 'coria', 'curat', 'curia', 'cymar']
Hint (most likely candidate or for maximizing information): sabot


Enter a word for guess 2: sabot


sabot 
 ⬛ 🟨 ⬛ ⬛ ⬛ 
[35, 5] possibilities: ['cigar', 'circa', 'cimar', 'curia', 'cymar']
Hint (most likely candidate or for maximizing information): cigar


Enter a word for guess 3: cigar


cigar 
 🟩 🟩 🟩 🟩 🟩 
Impressive


In [15]:
# Class that represents a Wordle application as a whole
class wordle:
    
    # Initialize
    def __init__(self):
        self.games = []
        self.streaks = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

    # Print all previously played games
    def history(self):
        for item in self.games:
            print(item)

    # Print streaks
    def records(self):
        return [ print(f"{key}: {value}") for key, value in self.streaks.items() ]
    
    # Update streaks
    def add_win(self, tries):
        self.streaks[tries] = self.streaks[tries] + 1

    # Reset streaks
    def reset(self):
        self.streaks = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

    # Play a Wordle game and save stats afterwards
    def play_wordle(self):
        new_game = game()
        
        # Choose Wordle number
        date = input("Enter the game number you want to play:") 
        if date.isdigit() and int(date) >= 0 and int(date) <= 2314:
            new_game.set_game(number = int(date))
        else:
            print("Invalid input, choosing a random game.")
            new_game.set_game()

        # Play game
        new_game.play()
        self.games.append(new_game)
        
        if(new_game.attempts < 6):
            self.add_win(new_game.attempts) # Game won
        else:
            self.reset() # Game lost, reset streaks

In [6]:
# Create a Wordle instance
my_session = wordle() 

# Data is re-initialized whenever this is run!

In [7]:
my_session.play_wordle() # Play a game of Wordle

print('\nGame history:\n') # View past games
my_session.history()

print('Streaks:') # View streaks
my_session.records()

cigar 
 🟩 🟩 🟩 🟩 🟩 
Genius

Game history:

cigar: guessed in 1 tries: 
['cigar']

Streaks:
1: 1
2: 0
3: 0
4: 0
5: 0
6: 0


[None, None, None, None, None, None]